In [1]:
import time
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [13]:
# 自定义函数 -----------------------------------------------------------------------------------------------------
def name_to_str(obj, namespace):
    return [name for name in namespace if namespace[name] is obj]

def find_index(index, pred_type):
    equal = np.empty(0)
    for i in np.arange(0, pred_type.size):
        count = index.count(pred_type[i])
        if count == 0:
            equal = np.append(equal, "False")
        else:
            equal = np.append(equal, "True")
    return pd.DataFrame({"equal":equal})

def result(test_x, test_y):
    # 预测
    pred = model.predict(test_x)
    truth_type = test_y.argmax(axis = 1)
    pred_type = pred.argmax(axis = 1)
    
    # test准确性
    single_accuracy = round(model.evaluate(test_x, test_y)[1], 3)
    print("\n★--- Single_accuracy:", single_accuracy)
    print("\n------------------------------------------------------------------")
    
    # 获取对应索引
    name = name_to_str(pred_x,globals())[0]
    if name == "x_test_normal": class_index = normal_index; total_index = normal_index
    if name == "x_test_speeding": class_index = speeding_index; total_index = aberrant_index
    if name == "x_test_cheating": class_index = cheating_index; total_index = aberrant_index
    if name == "x_test_r_guessing": class_index = r_guessing_index; total_index = aberrant_index
    # print(class_index, total_index)
    
    # class准确性
    class_df = find_index(class_index, pred_type)
    class_accuracy = round(class_df.value_counts()["True"] / class_df.size, 3)
    
    print("\n★--- Class_accuracy:", class_accuracy)
    print("\n------------------------------------------------------------------")
    
    # total准确性
    total_df = find_index(total_index, pred_type)
    total_accuracy = round(total_df.value_counts()["True"] / total_df.size, 3)
    print("\n★--- Total_accuracy:", total_accuracy)
    print("\n------------------------------------------------------------------")
    
    return single_accuracy, class_accuracy, total_accuracy

In [15]:
# 生成保存结果的数组 -----------------
normal_single_accuracy = np.empty(0)
speeding_single_accuracy = np.empty(0)
cheating_single_accuracy = np.empty(0)
r_guessing_single_accuracy = np.empty(0)

normal_class_accuracy = np.empty(0)
speeding_class_accuracy = np.empty(0)
cheating_class_accuracy = np.empty(0)
r_guessing_class_accuracy = np.empty(0)

normal_total_accuracy = np.empty(0)
speeding_total_accuracy = np.empty(0)
cheating_total_accuracy = np.empty(0)
r_guessing_total_accuracy = np.empty(0)

# 重复50次 (1:51) -----------------------------
print(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime()))
for rs in np.arange(1, 2):
    
    ########## 数据参数 ##########
    start_name = "Q_3_10_high_0.2"
    seed = str(rs)
    hasRT = "F"
    ##############################
    print("--------------------- running " + seed + " ---------------------")
    
    # 读入数据 ---------------------
    train_x_path = "./DATA2/" + seed + "s_" + start_name + "_train_x_" + hasRT + ".csv"
    train_y_path = "./DATA2/"  + seed + "s_" + start_name + "_train_y.csv"

    test_normal_x_path = "./DATA2/"  + seed + "s_" + start_name + "_test_normal_x_" + hasRT + ".csv"
    test_normal_y_path = "./DATA2/"  + seed + "s_" + start_name + "_test_normal_y.csv"
    test_cheating_x_path = "./DATA2/"  + seed + "s_" + start_name + "_test_cheating_x_" + hasRT + ".csv"
    test_cheating_y_path = "./DATA2/"  + seed + "s_" + start_name + "_test_cheating_y.csv"
    test_speeding_x_path = "./DATA2/"  + seed + "s_" + start_name + "_test_speeding_x_" + hasRT + ".csv"
    test_speeding_y_path = "./DATA2/"  + seed + "s_" + start_name + "_test_speeding_y.csv"
    test_r_guessing_x_path = "./DATA2/"  + seed + "s_" + start_name + "_test_r_guessing_x_" + hasRT + ".csv"
    test_r_guessing_y_path = "./DATA2/"  + seed + "s_" + start_name + "_test_r_guessing_y.csv"

    x_train = np.array(pd.read_csv(train_x_path).drop("Unnamed: 0", axis=1))
    y_train = np.array(pd.read_csv(train_y_path).drop("Unnamed: 0", axis=1))

    x_test_normal = np.array(pd.read_csv(test_normal_x_path).drop("Unnamed: 0", axis=1))
    y_test_normal = np.array(pd.read_csv(test_normal_y_path).drop("Unnamed: 0", axis=1))
    x_test_cheating = np.array(pd.read_csv(test_cheating_x_path).drop("Unnamed: 0", axis=1))
    y_test_cheating = np.array(pd.read_csv(test_cheating_y_path).drop("Unnamed: 0", axis=1))
    x_test_speeding = np.array(pd.read_csv(test_speeding_x_path).drop("Unnamed: 0", axis=1))
    y_test_speeding = np.array(pd.read_csv(test_speeding_y_path).drop("Unnamed: 0", axis=1))
    x_test_r_guessing = np.array(pd.read_csv(test_r_guessing_x_path).drop("Unnamed: 0", axis=1))
    y_test_r_guessing = np.array(pd.read_csv(test_r_guessing_y_path).drop("Unnamed: 0", axis=1))

    # 获取异常类索引 ---------------
    def get_same_element_index(ob_list, word):
        return [i for (i, v) in enumerate(ob_list) if v == word]

    normal_list = pd.read_csv(train_y_path).drop("Unnamed: 0", axis=1).columns.str.contains('normal').tolist()
    speeding_list = pd.read_csv(train_y_path).drop("Unnamed: 0", axis=1).columns.str.contains('speeding').tolist()
    cheating_list = pd.read_csv(train_y_path).drop("Unnamed: 0", axis=1).columns.str.contains('cheating').tolist()
    r_guessing_list = pd.read_csv(train_y_path).drop("Unnamed: 0", axis=1).columns.str.contains('r_guessing').tolist()

    normal_index = get_same_element_index(normal_list, True)
    aberrant_index = get_same_element_index(normal_list, False)
    speeding_index = get_same_element_index(speeding_list, True)
    cheating_index = get_same_element_index(cheating_list, True)
    r_guessing_index = get_same_element_index(r_guessing_list, True)
    

    # 训练集——测试集划分（5-5） ---
    x_train,x_val,y_train,y_val = train_test_split(x_train,y_train,train_size=0.5,random_state=666)
    print("x_train.shape:", x_train.shape)
    print("y_train.shape:", y_train.shape)
    
    ########## 模型参数 ##########
    layer_one = x_train.shape[1] - 1
    batch_size = int(x_train.shape[0] / 1000)
    learning_rate=0.0001
    epochs = 100
    ##############################
    
    # 构建模型 -----------------------
    model = Sequential()
    model.add(Dense(units = layer_one, input_dim = x_train.shape[1], activation = "relu"))
    model.add(Dense(units = y_train.shape[1], activation = "softmax"))
    
    adam = Adam(learning_rate = learning_rate)
    
    model.compile(optimizer = adam, loss = "categorical_crossentropy", metrics = ['accuracy'])
    
    # 训练模型 ------------------------
    H = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_val, y_val), workers=20, use_multiprocessing=True)
    
    # normal结果 ----------------------
    pred_x = x_test_normal
    pred_y = y_test_normal
    print(name_to_str(pred_x,globals()))
    single_accuracy, class_accuracy, total_accuracy = result(pred_x, pred_y)
    normal_single_accuracy = np.append(normal_single_accuracy, single_accuracy)
    normal_class_accuracy = np.append(normal_class_accuracy, class_accuracy)
    normal_total_accuracy = np.append(normal_total_accuracy, total_accuracy)
    
    # speeding结果 --------------------
    pred_x = x_test_speeding
    pred_y = y_test_speeding
    print(name_to_str(pred_x,globals()))
    single_accuracy, class_accuracy, total_accuracy = result(pred_x, pred_y)
    speeding_single_accuracy = np.append(speeding_single_accuracy, single_accuracy)
    speeding_class_accuracy = np.append(speeding_class_accuracy, class_accuracy)
    speeding_total_accuracy = np.append(speeding_total_accuracy, total_accuracy)

    # cheating结果 ---------------------
    pred_x = x_test_cheating
    pred_y = y_test_cheating
    print(name_to_str(pred_x,globals()))
    single_accuracy, class_accuracy, total_accuracy = result(pred_x, pred_y)
    cheating_single_accuracy = np.append(cheating_single_accuracy, single_accuracy)
    cheating_class_accuracy = np.append(cheating_class_accuracy, class_accuracy)
    cheating_total_accuracy = np.append(cheating_total_accuracy, total_accuracy)

    # r_guessing结果 -------------------
    pred_x = x_test_r_guessing
    pred_y = y_test_r_guessing
    print(name_to_str(pred_x,globals()))
    single_accuracy, class_accuracy, total_accuracy = result(pred_x, pred_y)
    r_guessing_single_accuracy = np.append(r_guessing_single_accuracy, single_accuracy)
    r_guessing_class_accuracy = np.append(r_guessing_class_accuracy, class_accuracy)
    r_guessing_total_accuracy = np.append(r_guessing_total_accuracy, total_accuracy)
    
print(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime()))

2022/11/12 22:26:02
--------------------- running 1 ---------------------
x_train.shape: (10000, 10)
y_train.shape: (10000, 32)
Epoch 1/100
1000/1000 [==============================] - 2s 2ms/step - loss: 3.3581 - accuracy: 0.0751 - val_loss: 3.2340 - val_accuracy: 0.0984
Epoch 2/100
1000/1000 [==============================] - 2s 2ms/step - loss: 3.1278 - accuracy: 0.0962 - val_loss: 3.0177 - val_accuracy: 0.1033
Epoch 3/100
1000/1000 [==============================] - 2s 2ms/step - loss: 2.9178 - accuracy: 0.1358 - val_loss: 2.8139 - val_accuracy: 0.1749
Epoch 4/100
1000/1000 [==============================] - 2s 2ms/step - loss: 2.7182 - accuracy: 0.2183 - val_loss: 2.6188 - val_accuracy: 0.2652
Epoch 5/100
1000/1000 [==============================] - 2s 2ms/step - loss: 2.5329 - accuracy: 0.2748 - val_loss: 2.4453 - val_accuracy: 0.3128
Epoch 6/100
1000/1000 [==============================] - 2s 2ms/step - loss: 2.3683 - accuracy: 0.3341 - val_loss: 2.2888 - val_accuracy: 0.3971
Ep

1000/1000 [==============================] - 2s 2ms/step - loss: 0.9008 - accuracy: 0.7227 - val_loss: 0.9045 - val_accuracy: 0.7158
Epoch 57/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.8982 - accuracy: 0.7217 - val_loss: 0.9021 - val_accuracy: 0.7159
Epoch 58/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.8956 - accuracy: 0.7228 - val_loss: 0.9000 - val_accuracy: 0.7165
Epoch 59/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.8933 - accuracy: 0.7228 - val_loss: 0.8978 - val_accuracy: 0.7165
Epoch 60/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.8909 - accuracy: 0.7238 - val_loss: 0.8958 - val_accuracy: 0.7172
Epoch 61/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.8887 - accuracy: 0.7238 - val_loss: 0.8939 - val_accuracy: 0.7167
Epoch 62/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.8866 - accuracy: 0.7235 - val_loss: 0.8919 - val_accuracy

In [ ]:
# 结果整理
res_dataFrame = pd.DataFrame({"normal_single_accuracy":normal_single_accuracy,
                              "speeding_single_accuracy":speeding_single_accuracy,
                              "cheating_single_accuracy":cheating_single_accuracy,
                              "r_guessing_single_accuracy":r_guessing_single_accuracy,
                              "normal_class_accuracy":normal_class_accuracy,
                              "speeding_class_accuracy":speeding_class_accuracy,
                              "cheating_class_accuracy":cheating_class_accuracy,
                              "r_guessing_class_accuracy":r_guessing_class_accuracy,
                              "normal_total_accuracy":normal_total_accuracy,
                              "speeding_total_accuracy":speeding_total_accuracy,
                              "cheating_total_accuracy":cheating_total_accuracy,
                              "r_guessing_total_accuracy":r_guessing_total_accuracy})
res_dataFrame